# Phase 15-8.5 — A/B Soak Analysis
Inputs: `.artifacts/wisdom_ab_runs.csv`

In [ ]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv('../.artifacts/wisdom_ab_runs.csv')
df.head()

In [ ]:
# summarize per (kappa,g0)
agg = df.groupby(['kappa','g0']).agg(
  eta_p95=('eta', lambda s: np.nanpercentile(s.dropna(),95)),
  S_mean=('S','mean'), H_min=('H','min'),
  G_mean=('Gstar','mean'), G_std=('Gstar','std'),
  bias_abs_mean=('bias', lambda s: s.abs().mean()),
  clamp_ratio=('clamp','mean'),
).reset_index()
agg

In [ ]:
# filter by success criteria
ok = (agg.S_mean>=0.03)&(agg.H_min>=0.02)&(agg.bias_abs_mean<=0.01)&(agg.G_mean>=0.6)&(agg.G_std<0.05)&(agg.clamp_ratio<0.10)
agg[ok].sort_values(['G_mean','S_mean'], ascending=[False,False])

In [ ]:
# plot example
for (k,g),gdf in df.groupby(['kappa','g0']):
    gdf=gdf[gdf.t.notna()]
    plt.figure(); plt.title(f'η over time (kappa={k}, g0={g})'); plt.plot(gdf.t, gdf.eta); plt.xlabel('t [s]'); plt.ylabel('eta'); plt.show()